In [55]:
from keras.models import Model
from keras.layers import Embedding, Dense, LSTM, Input, Reshape
from keras import optimizers
from keras.callbacks import ModelCheckpoint
import keras
import numpy as np
from data import Vocab
from batcher import Batcher

In [56]:
from hps import hps

In [75]:
embedding_init = keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=None)
embedding_layer = Embedding(hps['vocab_size'], hps['hidden_dim'], embeddings_initializer=embedding_init)

encoder_inputs = Input(shape=(None,), name='encoder_inputs')
encoder_lstm = LSTM(hps['hidden_dim'], return_state=True, name='encoder_lstm')
encoder_outputs, state_h, state_c = encoder_lstm(embedding_layer(encoder_inputs))
encoder_states = [state_h, state_c]
decoder_inputs = Input(shape=(None,), name='decoder_inputs')
decoder_lstm = LSTM(hps['hidden_dim'], return_state=True, return_sequences=True, name='decoder_lstm')
decoder_outputs, _, _ = decoder_lstm(embedding_layer(decoder_inputs), initial_state=encoder_states)
decoder_dense = Dense(hps['vocab_size'], activation='softmax', name='decoder_dense')
dense_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], dense_outputs)
adam = optimizers.Adam()
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_inputs = [Input(shape=(hps['hidden_dim'],)), Input(shape=(hps['hidden_dim'],))]
decoder_outputs, decoder_states = decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_state_inputs, [decoder_outputs] + decoder_states)

In [8]:
vocab = Vocab(hps['vocab_path'], hps['vocab_size'])









max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: perisic


In [82]:
  def make_feed_dict(batch, just_enc=False):
    """Make a feed dictionary mapping parts of the batch to the appropriate placeholders.
    Args:
      batch: Batch object
      just_enc: Boolean. If True, only feed the parts needed for the encoder.
    """
    feed_dict = {}
    feed_dict['enc_batch'] = batch.enc_batch
    feed_dict['dec_batch'] = batch.dec_batch
    feed_dict['target_batch'] = np.eye(hps['vocab_size'])[batch.target_batch]
    return feed_dict

In [72]:
mode = 'test'
train_batcher = Batcher(hps['data_path'] + '/{}.bin'.format(mode), vocab, hps, single_pass=True)

In [73]:
batch = train_batcher.next_batch()

In [83]:
feed_dict = make_feed_dict(batch)

In [84]:
X, Y = [feed_dict['enc_batch'], feed_dict['dec_batch']], feed_dict['target_batch']

In [85]:
X[1].shape

(16, 100)

In [102]:
def generate_batch(mode): #mode: train/test/val
    hps['mode'] = mode
    batcher = Batcher(hps['data_path'] + '/{}.bin'.format(mode), vocab, hps, single_pass=True)
#     batcher.fill_example_queue()
#     batcher.fill_batch_queue()
    while True:
        batch = batcher.next_batch()
        feed_dict = make_feed_dict(batch)
        yield [feed_dict['enc_batch'], feed_dict['dec_batch']], feed_dict['target_batch']


In [103]:
epochs = 10

In [ ]:
history = model.fit_generator(generator=generate_batch('test'), steps_per_epoch=hps['batch_size'],
                                   epochs=epochs,
                                   verbose=1, validation_data=generate_batch('val'), validation_steps=hps['batch_size'],
                                  use_multiprocessing=True,workers=6
                                   )

/Users/HelenWang/anaconda/envs/py36/lib/python3.6/site-packages/keras/engine/training_generator.py:44: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/10
15/16 [===========================>..] - ETA: 2:11 - loss: 9.1160 - acc: 0.8623

/Users/HelenWang/anaconda/envs/py36/lib/python3.6/site-packages/keras/engine/training_generator.py:272: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


16/16 [==============================] - 4750s 297s/step - loss: 8.9724 - acc: 0.8637 - val_loss: 6.4070 - val_acc: 0.8977
Epoch 2/10
 2/16 [==>...........................] - ETA: 21:44 - loss: 6.1930 - acc: 0.9062

In [86]:
model.fit(X, Y, batch_size=hps['batch_size'], epochs=10,)

Epoch 1/10
16/16 [==============================] - 29s 2s/step - loss: 10.8203 - acc: 0.0000e+00
Epoch 2/10
16/16 [==============================] - 19s 1s/step - loss: 10.8122 - acc: 0.4050
Epoch 3/10
16/16 [==============================] - 16s 1s/step - loss: 10.8036 - acc: 0.9500
Epoch 4/10
16/16 [==============================] - 19s 1s/step - loss: 10.7947 - acc: 0.9500
Epoch 5/10
16/16 [==============================] - 18s 1s/step - loss: 10.7838 - acc: 0.9500
Epoch 6/10
16/16 [==============================] - 17s 1s/step - loss: 10.7690 - acc: 0.9500
Epoch 7/10
16/16 [==============================] - 16s 973ms/step - loss: 10.7493 - acc: 0.9400
Epoch 8/10
16/16 [==============================] - 16s 987ms/step - loss: 10.7219 - acc: 0.9306
Epoch 9/10
16/16 [==============================] - 15s 911ms/step - loss: 10.6823 - acc: 0.9200
Epoch 10/10
16/16 [==============================] - 14s 894ms/step - loss: 10.6230 - acc: 0.9013
